# Medium Green - DP

## 最长子序列问题

#### 5. 最长回文子串 Longest Palindromic Substring

In [ ]:
"""解法一：暴力法，枚举所有子串，并逐一判断 O(N**3), O(1)"""
class Solution:
    def longestPalindrome(self, s: str) -> str:
        # 特判
        lens = len(s)
        if lens < 2:
            return s 
        # 枚举所有长度大于等于2的子串，然后判断哪些是回文子串，如果是回文串，那就比较这个回文串的长度是不是最长的
        maxLen = 1
        res = ''
        for i in range(lens-1):
            for j in range(i+1, lens):
                if self.isPalin(s[i:j+1]):
                    if j+1-i > maxLen:
                        maxLen = j+1-i 
                        res = s[i:j+1]
        return res 
    
    def isPalin(self, s: str) -> bool:
        left, right = 0, len(s)-1
        while left < right:
            if s[left] != s[right]:
                return False
            left += 1
            right -= 1
        return True

In [ ]:
"""解法二：中心扩散法，center spread
暴力法采用双指针两边夹，验证是否是回文子串，时间复杂度比较高，除了枚举字符串的左右边界以外，比较容易想到的是枚举可能出现的回文子串的“中心位置”
从“中心位置”尝试尽可能扩散出去，得到一个回文串。
因此，中心扩散法的思路是：遍历每一个索引，以这个索引为中心，利用“回文串”中心对称的特点，往两边扩散，看最多能扩散多远。
枚举“中心位置”时间复杂度为 O(N)，从“中心位置”扩散得到“回文子串”的时间复杂度为 O(N)，因此时间复杂度可以降到 O(N^2)。
在这里要注意一个细节：回文串在长度为奇数和偶数的时候，“回文中心”的形式是不一样的。
奇数回文串的“中心”是一个具体的字符，例如：回文串 "aba" 的中心是字符 "b"；
偶数回文串的“中心”是位于中间的两个字符的“空隙”，例如：回文串串 "abba" 的中心是两个 "b" 中间的那个“空隙”。"""
class Solution:
    def longestPalindrome(self, s: str) -> str: 
        lens = len(s)
        if lens < 2:
            return s 
        
        maxLen = 1
        res = s[0]   # 初始化为s[0]的原因是如果完全找不到一个回子串至少要输出第一个元素
        for i in range(lens):
            palin_odd, lens_odd = self._central_spread(s, i, i)
            palin_even, lens_even = self._central_spread(s, i, i+1)
            
            (currMaxPalin, currMaxLen) = (palin_odd, lens_odd) if lens_odd > lens_even else (palin_even, lens_even)
            if currMaxLen > maxLen:
                maxLen = currMaxLen
                res = currMaxPalin
        return res
    # return the longest palin starting from left, right, and the correponding lenth
    def _central_spread(self, s, left, right):   
        lens = len(s)
        i, j = left, right
        while i >= 0 and j < lens and s[i]==s[j] :
            i -= 1
            j += 1
        return s[i+1:j], j-i-1

In [ ]:
"""方法三：动态规划（推荐）
推荐理由：暴力解法太 naive，中心扩散不普适，Manacher 就更不普适了，是专门解这个问题的方法。而用动态规划我认为是最有用的，可以帮助你举一反三的方法。
解决这类 “最优子结构” 问题，可以考虑使用 “动态规划”：
1、定义 “状态”；
2、找到 “状态转移方程”。
记号说明： 下文中，使用记号 s[l, r] 表示原始字符串的一个子串，l、r 分别是区间的左右边界的索引值，使用左闭、右闭区间表示左右边界可以取到。举个例子，当 s = 'babad' 时，s[0, 1] = 'ba' ，s[2, 4] = 'bad'。
1、定义 “状态”，这里 “状态”数组是二维数组。
dp[l][r] 表示子串 s[l, r]（包括区间左右端点）是否构成回文串，是一个二维布尔型数组。即如果子串 s[l, r] 是回文串，那么 dp[l][r] = true。
2、找到 “状态转移方程”。
首先，我们很清楚一个事实：
1、当子串只包含 1 个字符，它一定是回文子串；
2、当子串包含 2 个以上字符的时候：如果 s[l, r] 是一个回文串，例如 “abccba”，那么这个回文串两边各往里面收缩一个字符（如果可以的话）的子串 s[l + 1, r - 1] 也一定是回文串，即：如果 dp[l][r] == true 成立，一定有 dp[l + 1][r - 1] = true 成立。
根据这一点，我们可以知道，给出一个子串 s[l, r] ，如果 s[l] != s[r]，那么这个子串就一定不是回文串。如果 s[l] == s[r] 成立，就接着判断 s[l + 1] 与 s[r - 1]，这很像中心扩散法的逆方法。
事实上，当 s[l] == s[r] 成立的时候，dp[l][r] 的值由 dp[l + 1][r - l] 决定，这一点也不难思考：当左右边界字符串相等的时候，整个字符串是否是回文就完全由“原字符串去掉左右边界”的子串是否回文决定。但是这里还需要再多考虑一点点：“原字符串去掉左右边界”的子串的边界情况。
1、当原字符串的元素个数为 3 个的时候，如果左右边界相等，那么去掉它们以后，只剩下 1 个字符，它一定是回文串，故原字符串也一定是回文串；也就是说当r - l <= 2 且 s[left]==[right] 时，那么从left到right肯定是回文串， meaning dp[left][right]=True
综上，如果一个字符串的左右边界相等，以下二者之一成立即可：
1、去掉左右边界以后的字符串不构成区间，即“ s[l + 1, r - 1] 至少包含两个元素”的反面，即 r - l <= 2；
2、去掉左右边界以后的字符串是回文串，具体说，它的回文性决定了原字符串的回文性。
于是整理成“状态转移方程” dp[l, r] = (s[l] == s[r] and (r - l <= 2 or dp[l + 1, r - 1]))
编码实现细节：因为要构成子串 l 一定小于等于 r ，我们只关心 “状态”数组“上三角”的那部分取值。理解上面的“状态转移方程”中的 (r - l <= 2 or dp[l + 1, r - 1]) 这部分是关键，因为 or 是短路运算，因此，如果收缩以后不构成区间，那么就没有必要看继续 dp[l + 1, r - 1] 的取值。
时间复杂度：O(N^{2}); 空间复杂度：O(N^{2})，二维 dp 问题，一个状态得用二维有序数对表示，因此空间复杂度是 O(N^{2})。"""
class Solution:
    def longestPalindrome(self, s: str) -> str:
        lens = len(s)
        if lens <= 1:
            return s 
        
        maxLen = 1
        res = s[0]   # 初始化为s[0]的原因是如果完全找不到一个回子串至少要输出第一个元素
        dp = [[False]*lens for _ in range(lens)]
        for right in range(1, lens):
            for left in range(right):
                if s[left]==s[right] and (dp[left+1][right-1] or right-left <= 2):
                    dp[left][right] = True 
                    if right-left+1 > maxLen:
                        maxLen = right-left+1
                        res = s[left:right+1]
        return res

In [ ]:
"""方法三：动态规划 + 空间优化  时间复杂度：O(N^2); 空间复杂度：O(N)
对于方法二中动态规划的状态转移方程，我们发现 dp(i, j) 只和 dp(i+1, j) 和 dp(i, j-1) 有关，即在计算第 i 行的 dp 值时，只有该行与第 i + 1 行是有用的，意思其实是说一行只取一个数，而不是一行需要取不同列上的不同数。因此我们可以将空间优化至一维。"""
class Solution:
    def PredictTheWinner(self, nums: List[int]) -> bool:
        lens = len(nums)
        dp = [0 for _ in range(lens)]
        for row in range(lens-1, -1, -1):
            dp[row] = nums[row]
            for col in range(row+1, lens):
                dp[col] = max(nums[row]-dp[col], nums[col]-dp[col-1])
        return dp[lens-1] >= 0

#### 516. 最长回文子序列
给定一个字符串s，找到其中最长的回文子序列。可以假设s的最大长度为1000。
示例 1:
输入:
"bbbab"
输出:
4
一个可能的最长回文子序列为 "bbbb"。

In [ ]:
"""解法一：递归, O(?)"""
class Solution:
    def longestPalindromeSubseq(self, s: str) -> int:
        def _helper_(s):
            if len(s) <= 1:
                return len(s)
            if s[0] == s[-1]:
                return _helper_(s[1:-1]) + 2
            else:
                return max(_helper_(s[1:]), _helper_(s[:-1]))
        return _helper_(s)

In [ ]:
"""解法二：下面是带memo的递归形式的解法，memo数组这里起到了一个缓存已经计算过了的结果，这样能提高运算效率，使其不会time limit ecceeded TLE
O(N^2), O(N^2)"""
class Solution:
    def longestPalindromeSubseq(self, s: str) -> int:
        lens = len(s)
        memo = [[-1]*lens for _ in range(lens)]
        return self._helper_(s, 0, lens-1, memo)
        
    def _helper_(self, s, left, right, memo):
        if left > right:
            return 0
        if left == right:
            return 1
        if memo[left][right] != -1:
            return memo[left][right]   # 只要memo[left][right]被计算过，就直接调用memo[left][right]，就不用再计算一遍了
        if s[left] == s[right]:
            memo[left][right] = self._helper_(s, left+1, right-1, memo) + 2
        else:
            memo[left][right] = max(self._helper_(s, left+1, right, memo), self._helper_(s, left, right-1, memo))
            
        return memo[left][right]

In [ ]:
"""解法三：参考递归法的思路尝试一下DP； O(N^2), O(N^2)
动态规划四要素：
1. dp[i][j]表示字符串s[i:j]的最长回文子序列的长度
2. 递推关系：dp[i][j] = dp[i+1][j-1]+2 当s[i]=s[j]
dp[i][j]=max(dp[i+1][j],dp[i][j-1]) 当s[i]!=s[j] 取s[i+1..j] 和s[i..j-1]中最长的 由于dp[i][j]需要dp[i+1][j]所以需要i逆序枚举s的长度，而又因为dp[i][j]需要dp[i][j-1],所以要求j是递增的，这样在保证求解dp[i][j]时,dp[i][j-1]肯定已经求解过了，又由于dp[i][j]表示从i到j的字符串的最长回文子序列的长度，所以要保证j大于i，所以j从i+1开始遍历。这样就可以保证这样可以保证每个子问题都已经算好了。
3. 初始化dp[i][i] = 1 单个字符的最长回文序列是 1
4. 结果：dp[0][lens-1]"""
class Solution:
    def longestPalindromeSubseq(self, s: str) -> int:
        lens = len(s)
        dp = [[0]*lens for _ in range(lens)]
        for i in range(lens):
            dp[i][i] = 1
        for i in range(lens-1, -1, -1):
            for j in range(i+1, lens):
                if s[i] == s[j]:
                    dp[i][j] = dp[i+1][j-1] + 2
                else:
                    dp[i][j] = max(dp[i+1][j], dp[i][j-1])
        return dp[0][lens-1]

In [ ]:
"""解法四：优化成一位数组 DP； O(N^2), O(N)
然后在遍历过程中，你会发现，你只需要j位置的数据，不需要之前的，所以可以把二维变成两个一维数组。要好好体会理解"""
class Solution:
    def longestPalindromeSubseq(self, s: str) -> int:
        lens = len(s)
        if lens <= 1:
            return lens
        dp = [1]*lens
        for i in range(lens-1, -1, -1):
            prev = 0
            for j in range(i+1, lens):
                temp = dp[j]
                if s[i] == s[j]:
                    dp[j] = prev + 2
                prev = max(temp, prev)
        return max(dp)

In [ ]:
一般来说，这类问题都是让你求一个最长子序列，因为最短子序列就是一个字符嘛，没啥可问的。一旦涉及到子序列和最值，那几乎可以肯定，考察的是动态规划技巧，时间复杂度一般都是 O(n^2)。
模板1：
lens = len(S)
dp = [[]*lens for _ in range(lens)]

for left in range(lens-1, -1, -1):
    for right in range(i+1, lens):
        if -----:
            dp[left][right] = 最值(dp[left+1][right-1].......)

模板2：
lens = len(S)
dp = [0]*lens

for i in range(lens-1, -1, -1):
    for j in range(i+1, lens):
        if -----:
            dp[j] = 最值(dp[j], dp[j-1]+...)

#### 300. 最长上升子序列 Longest Increasing Subsequence
给定一个无序的整数数组，找到其中最长上升子序列的长度。

In [ ]:
"""方法一：暴力法
时间复杂度：O(2^n)递归树的大小将为 2^n
算法：
最简单的方法是找到所有增加的子序列，然后返回最长增加的子序列的最大长度。为了做到这一点，我们使用递归函数 \text length of lislengthoflis 返回从当前元素（对应于 curposcurpos）开始可能的 lis 长度（包括当前元素）。在每个函数调用中，我们考虑两种情况：
当前元素大于包含在 lis 中的前一个元素。在这种情况下，我们可以在 lis 中包含当前元素。因此，我们通过将其包含在内，得出了 lis 的长度。此外，我们还通过不在 lis 中包含当前元素来找出 lis 的长度。因此，当前函数调用返回的值是两个长度中的最大值。
当前元素小于包含在 lis 中的前一个元素。在这种情况下，我们不能在 lis 中包含当前元素。因此，我们只通过不在 lis 中包含当前元素（由当前函数调用返回）来确定 lis 的长度。"""
class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        return self._lengthOfLIS_Helper(nums, -float("inf"), 0)
    
    def _lengthOfLIS_Helper(self, nums: List[int], prevVal: int, curr: int) -> int:
        if curr == len(nums):
            return 0
        taken, noTaken = 0, 0
        if nums[curr] > prevVal:
            taken = 1 + self._lengthOfLIS_Helper(nums, nums[curr], curr+1)
        # 注意把noTaken放在else里面是错的，因为即使nums[curr]>prevVal也可以选择不take，不take当前那个数说不定更好，eg:[1,2,9,3,4,5],这里如果take 9，那最长的长度是3，不take 9的话最长的长度可以是5
        noTaken = self._lengthOfLIS_Helper(nums, prevVal, curr+1)

        return max(taken, noTaken)

In [ ]:
"""方法二：带记忆的递归 O(N^2); O(N^2)
算法：在前面的方法中，许多递归调用必须使用相同的参数进行一次又一次的调用。通过将为特定调用获得的结果存储在二维记忆数组 memo 中，可以消除这种冗余。memo[i][j] 表示使用 nums[i] 作为上一个被认为包含/不包含在 lis 中的元素的 lis 可能的长度，其中 nums[j] 作为当前被认为包含/不包含在 lis 中的元素。这里，nums 表示给定的数组。"""
class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        lens = len(nums)
        memo = [[0]*(lens) for _ in range(lens)]
        return self._lengthOfLIS_Helper(nums, -1, 0, memo)

    def _lengthOfLIS_Helper(self, nums: List[int], prev: int, curr: int, memo: List[int]) -> int:
        if curr == len(nums):
            return 0
        taken, noTaken = 0, 0
        if memo[prev+1][curr] > 0:
            return memo[prev+1][curr]   # 这里判断只要memo[prev+1][curr]只要计算过，就直接调用不再计算了，从而降低了运算的复杂度。
        if prev < 0 or nums[curr] > nums[prev]:
            taken = 1 + self._lengthOfLIS_Helper(nums, curr, curr+1, memo)
        noTaken = self._lengthOfLIS_Helper(nums, prev, curr+1, memo)
        memo[prev+1][curr] = max(taken, noTaken)
        return memo[prev+1][curr]

In [ ]:
"""方法三：动态规划  时间复杂度：O(n^2)。有两个 n 的循环。空间复杂度：O(n)，用了大小为 n 的矩阵 dp。
算法：dp[i]表示到第i个数据为止最大的长度
递推关系：if j>i and nums[j]>nums[i]: dp[j]=max[dp[j], dp[i]+1]
初始条件：dp[0] = 1"""
class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        lens = len(nums)
        if lens == 0:
            return 0
        dp = [1]*lens
        for i in range(lens-1):
            for j in range(i+1, lens):
                if nums[j] > nums[i]:
                    dp[j] = max(dp[j], dp[i]+1)
        return max(dp)
"""方法四：将dp的方法优化，使用二分查找。时间复杂度 O(NlogN) ： 遍历 nums 列表需 O(N)，在每个 nums[i]n 二分法需 O(logN)。
空间复杂度 O(N) ： dp 列表占用线性大小额外空间。"""

#### 673. 最长递增子序列的个数
给定一个未排序的整数数组，找到最长递增子序列的个数。
输入: [1,3,5,4,7]
输出: 2
解释: 有两个最长递增子序列，分别是 [1, 3, 4, 7] 和[1, 3, 5, 7]。

In [ ]:
class Solution:
    def findNumberOfLIS(self, nums: List[int]) -> int:
        lens = len(nums)
        if lens==0:
            return 0
        dp = [1]*lens   # longest length ending in nums[i]
        cnt = [1]*lens  # number of longest ending in nums[i]
        for i in range(lens-1):
            for j in range(i+1, lens):
                if nums[j] > nums[i]:
                    if dp[j] <= dp[i]:
                        dp[j] = 1 + dp[i]
                        cnt[j] = cnt[i]
                    else:
                        if dp[i]+1 == dp[j]:   # 说明dp[j]是dp[i]的下一个递增的元素
                            cnt[j] += cnt[i]
        maxLens = max(dp)
        res = 0
        for i, num in enumerate(dp):
            if num == maxLens:
                res += cnt[i]
        return res

#### 1027. Longest Arithmetic Sequence
给定一个整数数组 A，返回 A 中最长等差子序列的长度。

In [ ]:
"""暴力法：O(N^3), O(N)
思路：
等差数列 其实在 给定前2个数之后，就能求出整个等差数列(虽说是无限长的)。
既然如此，那就不断地 固定等差数列的 前2个数，去判断下1个理论值 是否在数组A[]中 且 在数组A[]中的下标 要 > 等差数列的最后1个实际值的下标(即理论值的 上1个值的下标)。
具体实现：
构建1个HashMap：数组A[]中的值作为key，值所对应的下标 去组成1个ArrayList 作为value。"""
class Solution:
    def longestArithSeqLength(self, A: List[int]) -> int:
        lens = len(A)
        dictA = {}
        for i, num in enumerate(A):
            if num in dictA.keys():
                dictA[num].append(i)      # 注意这里可能会有不同的下标对应num值
            else:
                dictA[num] = [i]
        zeroCnt, res = 1, 2
        for i in range(lens-1):
            for j in range(i+1, lens):
                diff = A[j]-A[i]
                if diff == 0:
                    zeroCnt = max(len(dictA[A[i]]), zeroCnt)    # 数组中可能会有多个重复的数，如[2,2,2,3,4,4,4,4]，所以这里用max
                else:
                    temp = j
                    cnt = 2
                    for k in range(j+1, lens):
                        if A[k]-A[temp] == diff:
                            cnt += 1
                            temp = k
                    res = max(res, cnt)
        return max(res, zeroCnt)

In [ ]:
"""实际上这个问题和之前问题Leetcode 300：最长上升子序列（最详细的解法！！！）非常类似，我们可以使用动态规划来做。我们知道对于等差数列有三种情况
上升; 下降; 不变
我们可以先定义函数f(x)表示以x结尾的最长等差数列的长度，那么对于第一种情况就会有下面式子
f(x)=1+f(y) if A[x]>A[y] and A[x]−A[y]=v
解释一下这个式子，我们假设等差数列的差为dis=v，对于y∈[0,x)，存在A[x]>A[y]并且A[x]−A[y]=v，那么f(x)=1+f(y)。同理对于下降和不变，我们同样可以得到下面两个式子
f(x)=1+f(y) if A[x]<A[y] and A[y]−A[x]=v
f(x)=1+f(y) if A[x]=A[y] and A[x]−A[y]=0
其实你会发现，当我们不考虑x和y的前后顺序的时候，上面三个式子表述的是一个式子。"""
class Solution:
    def longestArithSeqLength(self, A: List[int]) -> int:
        lens = len(A)
        # 构建一个二维字典，collections.defaultdict(int)的默认值是0，可以通过collections.defaultdict(lambda: 1)把默认值改成1
        dp = [collections.defaultdict(lambda: 1) for _ in range(lens)]   # 在每个位置上，以字典结构保存该位置元素与前面每个位置上元素的差值,这个差值就是字典的key，对应该差值的数列长度是字典的value。
        res = 1
        for i in range(lens-1):
            for j in range(i+1, lens):
                diff = A[j]-A[i]
                # 这里的二维字典dp[j][diff]表示以j作为键值1，键值1对应的value是一个嵌套的dict，这个嵌套的dict里面可能有很多key-val pair，这个嵌套的dict的key是diff，value是该j下标对应该差值diff的数列长度
                dp[j][diff] = max(dp[i][diff]+1, dp[j][diff])   ###### 要明白这里为什么不直接写dp[j][diff] = dp[i][diff]+1
                res = max(dp[j][diff], res)
        return res
"""O(N^2), O(N^2)"""

#### 873. 最长的斐波那契数列的长度
给定一个严格递增的正整数数组形成序列，找到 A 中最长的斐波那契式的子序列的长度。如果一个不存在，返回  0 。
输入: [1,3,7,11,12,14,18]
输出: 3
解释:
最长的斐波那契式子序列有：
[1,11,12]，[3,11,14] 以及 [7,11,18] 。

In [ ]:
"""想法很简单，就是每次从A中找前两个数，然后查看后续符合斐波那契的数在A中有没有。复杂度O(n^2 * logn) 空间复杂度O(1)"""
class Solution:
    def lenLongestFibSubseq(self, A: List[int]) -> int:
        lens = len(A)
        maxCnt = 2
        for i in range(lens-1):
            for j in range(i+1, lens):
                sums = A[i]+A[j]
                cnt = 2
                tempI, tempJ = A[i], A[j]
                temp = j
                while self.binarySearch(A[temp:], sums) >= 0:   # 由于数组是严格递增的，所以这里可以用二分法
                    cnt += 1
                    tempI, tempJ, temp = tempJ, sums, self.binarySearch(A[temp:], sums)
                    sums = tempI + tempJ
                maxCnt = max(maxCnt, cnt)
        return 0 if maxCnt < 3 else maxCnt
        
    def binarySearch(self, A: List[int], target: int) -> int:
        left, right = 0, len(A)-1
        while left <= right:
            mid = left+(right-left)//2
            if A[mid] == target:
                return mid
            elif A[mid] > target:
                right = mid - 1
            else:
                left = mid + 1
        return -1

In [ ]:
"""动态规划：dp[i][j]表示***以A[i]和A[j]***为结尾的子序列的长度。
初始条件：dp[i][i]=1; 递推关系：if j>i, dp[i][dict[A[i]+A[j]]] = dp[i][j]+1
与1027. Longest Arithmetric Sequence比较像，关键是理解dp[i][j]表示什么"""
class Solution:
    def lenLongestFibSubseq(self, A: List[int]) -> int:
        lens = len(A)
        dp = [[2]*lens for _ in range(lens)]
        
        dictA = collections.defaultdict(int)  # 用defuultdict即使有重复的num也可以分开存放
        for i, num in enumerate(A):
            dictA[num] = i
            
        res = 2
        for i in range(lens-1):
            for j in range(i+1, lens):
                dp[i][j] = max(dp[i][j], 2)
                sums = A[i] + A[j]
                if sums in dictA:
                    index = dictA[sums]
                    # 想想为什么dp[i][index] = max(dp[i][j] + 1, dp[i][index])是错的，想想dp[i][j]的定义是表示以***A[i]和A[j]***为结尾的子序列的长度
                    dp[j][index] = max(dp[j][index], dp[i][j]+1)
                    res = max(res, dp[j][index])
        return 0 if res < 3 else res

#### 801. 使序列递增的最小交换次数
我们有两个长度相等且不为空的整型数组 A 和 B 。
我们可以交换 A[i] 和 B[i] 的元素。注意这两个元素在各自的序列中应该处于相同的位置。
在交换过一些元素之后，数组 A 和 B 都应该是严格递增的（数组严格递增的条件仅为A[0] < A[1] < A[2] < ... < A[A.length - 1]）。
给定数组 A 和 B ，请返回使得两个数组均保持严格递增状态的最小交换次数。假设给定的输入总是有效的。
示例:
输入: A = [1,3,5,4], B = [1,2,3,7]
输出: 1
解释: 
交换 A[3] 和 B[3] 后，两个数组如下:
A = [1, 3, 5, 7] ， B = [1, 2, 3, 4]
两个数组均为严格递增的。

In [ ]:
"""https://www.cnblogs.com/grandyang/p/9311385.html
和714.买卖股票的最佳时机很像，都有交换和不交换两种情况
我们可以优化上面解法的空间复杂度，由于当前位置的值只跟前一个位置相关，所以我们并不需要保存整个数组，用四个变量来分别表示当前位置和前一个位置的各两种状态，可以实现同样的效果，参见代码如下："""
class Solution:
    def minSwap(self, A: List[int], B: List[int]) -> int:
        lens = len(A)
        swap_prev = 1
        noSwap_prev = 0
        for i in range(1, lens):
            swap_curr, noSwap_curr = float("inf"), float("inf")
            if A[i] > A[i-1] and B[i] > B[i-1]:
                swap_curr = min(swap_curr, swap_prev + 1)
                noSwap_curr = min(noSwap_curr, noSwap_prev)
            if A[i] > B[i-1] and B[i] > A[i-1]:
                swap_curr = min(swap_curr, noSwap_prev+1)
                noSwap_curr = min(noSwap_curr, swap_prev)
            swap_prev, noSwap_prev = swap_curr, noSwap_curr
        return min(swap_curr, noSwap_curr)

#### 1143. 最长公共子序列
给定两个字符串 text1 和 text2，返回这两个字符串的最长公共子序列。
一个字符串的 子序列 是指这样一个新的字符串：它是由原字符串在不改变字符的相对顺序的情况下删除某些字符（也可以不删除任何字符）后组成的新字符串。
例如，"ace" 是 "abcde" 的子序列，但 "aec" 不是 "abcde" 的子序列。两个字符串的「公共子序列」是这两个字符串所共同拥有的子序列。
若这两个字符串没有公共子序列，则返回 0。
示例 1:
输入：text1 = "abcde", text2 = "ace" 
输出：3  
解释：最长公共子序列是 "ace"，它的长度为 3。

In [ ]:
"""解法一：二维DP：Try dynamic programming. DP[i][j] represents the longest common subsequence of text1[0 ... i] & text2[0 ... j].
DP[i][j] = DP[i - 1][j - 1] + 1 , if text1[i] == text2[j] DP[i][j] = max(DP[i - 1][j], DP[i][j - 1]) , otherwise"""
class Solution:
    def longestCommonSubsequence(self, text1: str, text2: str) -> int:
        len_1, len_2 = len(text1), len(text2)
        if len_1 == 0 or len_2 == 0:
            return 0
        dp = [[0]*(len_2+1) for _ in range(len_1+1)]
        for i in range(1, len_1+1):
            for j in range(1, len_2+1):
                if text1[i-1] == text2[j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                else:
                    dp[i][j] = max(dp[i-1][j], dp[i][j-1])
        return dp[-1][-1]

#### 718. 最长重复子数组
给两个整数数组 A 和 B ，返回两个数组中公共的、长度最长的子数组的长度。
示例 1:
输入:
A: [1,2,3,2,1]
B: [3,2,1,4,7]
输出: 3
解释: 
长度最长的公共子数组是 [3, 2, 1]。

In [ ]:
"""方法一：暴力法O(N^3)。实际这道题就是求 Longest Common Substring 的问题了。最暴力的方法就是遍历A中的每个位置，把每个位置都当作是起点进行和B从开头比较，每次A和B都同时前进一个，假如相等，则计数器会累加1，不相等的话，计数器会重置为0，每次用计数器 cnt 的长度来更新结果 res。"""
class Solution:
    def findLength(self, A: List[int], B: List[int]) -> int:
        lensA, lensB = len(A), len(B)
        if lensA == 0 or lensB == 0:
            return 0

        res = 0
        for i in range(lensA):
            for j in range(lensB):
                cnt = 0
                m, n = i, j
                while m < lensA and n < lensB and A[m] == B[n]:
                    m += 1
                    n += 1
                    cnt += 1
                res = max(res, cnt)
        return res

In [ ]:
"""方法二：Use dynamic programming. dp[i][j] will be the answer for inputs A[:i], B[:j]."""
class Solution:
    def findLength(self, A: List[int], B: List[int]) -> int:
        lensA, lensB = len(A), len(B)
        if lensA == 0 or lensB == 0:
            return 0
        dp = [[0]*(lensB+1) for _ in range(lensA+1)]
        res = 0
        for i in range(lensA):
            for j in range(lensB):
                if A[i] == B[j]:
                    dp[i+1][j+1] = dp[i][j] + 1    # 貌似有逻辑漏洞，A[i] == B[j]出现的前后顺序可以不管吗？
                    res = max(res, dp[i+1][j+1])
        return res

### 64. 最小路径和
给定一个包含非负整数的 m x n 网格，请找出一条从左上角到右下角的路径，使得路径上的数字总和为最小。
说明：每次只能向下或者向右移动一步。
示例:
输入:
[
  [1,3,1],
  [1,5,1],
  [4,2,1]
]
输出: 7
解释: 因为路径 1→3→1→1→1 的总和最小。

In [ ]:
class Solution:
    def minPathSum(self, grid: List[List[int]]) -> int:
        if not grid:
            return 0
        len_row, len_col = len(grid), len(grid[0])

        if len_row == 1:
            return sum(grid[0][i] for i in range(len_col))
        if len_col == 1:
            return sum(grid[0][i] for i in range(len_row))
        
        dp = [[0]*len_col for _ in range(len_row)]
        dp[0][0] = grid[0][0]
        # 注意要先更新两个边的部分，因为每次只能向下或者向右移动一步， 所以可以更新
        for i in range(1, len_row):
            dp[i][0] = dp[i-1][0] + grid[i][0]
        for j in range(1, len_col):
            dp[0][j] = dp[0][j-1] + grid[0][j]
        for i in range(1, len_row):
            for j in range(1, len_col):
                dp[i][j] = min(dp[i-1][j], dp[i][j-1]) + grid[i][j]
                
        return dp[len_row-1][len_col-1]
"""进阶：可否更新为一维甚至零维dp算法"""

### 1049. 最后一块石头的重量
有一堆石头，每块石头的重量都是正整数。
每一回合，从中选出任意两块石头，然后将它们一起粉碎。假设石头的重量分别为 x 和 y，且 x <= y。那么粉碎的可能结果如下：
如果 x == y，那么两块石头都会被完全粉碎；
如果 x != y，那么重量为 x 的石头将会完全粉碎，而重量为 y 的石头新重量为 y-x。
最后，最多只会剩下一块石头。返回此石头最小的可能重量。如果没有石头剩下，就返回 0。

In [ ]:
"""由于石头拿走还能放回去，因此可以简单地把所有石头看作两堆
假设总重量为 sum, 则问题转化为背包问题：
如何使两堆石头总重量接近 sum / 2
dp[i] 背包容量限制为 i 时能够装载的最大石块总重量
curStone 存在最优解 dp[i] 时需要考虑的下一块石块重量"""
class Solution:
    def lastStoneWeightII(self, stones: List[int]) -> int:
        lens = len(stones)
        maxCapacity = sum(stones)//2
        dp = [0] * (maxCapacity+1)
        for i in range(lens):
            currStone = stones[i]
            for j in range(maxCapacity, currStone-1, -1):
                dp[j] = max(dp[j], dp[j-currStone] + currStone)
        return sum(stones) - 2 * dp[maxCapacity]

### 322. 零钱兑换
给定不同面额的硬币 coins 和一个总金额 amount。编写一个函数来计算可以凑成总金额所需的最少的硬币个数。如果没有任何一种硬币组合能组成总金额，返回 -1。
输入: coins = [1, 2, 5], amount = 11
输出: 3 
解释: 11 = 5 + 5 + 1

In [ ]:
"""方法一：自下而上，时间复杂度：O(S*n)。空间复杂度：O(S)，dp 使用的空间。"""
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        # dp[i]表示amount = i时最少的组合方法数
        # 递推关系是dp[i] = dp[i-coin]+1
        dp = [float("inf")] * (amount+1)
        dp[0] = 0
        for coin in coins:
            for i in range(coin, amount+1):
                dp[i] = min(dp[i], dp[i-coin]+1)
        return -1 if dp[amount]==float("inf") else dp[amount]
"""还有BFS,DFS的算法，后面可以多思考一下"""

### 714. The best time to buy and sell stock with transaction fee
输入: prices = [1, 3, 2, 8, 4, 9], fee = 2
输出: 8
解释: 能够达到的最大利润:  
在此处买入 prices[0] = 1
在此处卖出 prices[3] = 8
在此处买入 prices[4] = 4
在此处卖出 prices[5] = 9
总利润: ((8 - 1) - 2) + ((9 - 4) - 2) = 8.

In [ ]:
"""提示：Consider the first K stock prices. At the end, the only legal states are that you don't own a share of stock, or that you do. Calculate the most profit you could have under each of these two cases.
定义dp[i][0]为i天后手里有股票的最大收益，dp[i][1]为i天后手里没有股票的最大收益
递推关系：dp[i][0] = max(dp[i-1][1]-prices[i], dp[i-1][0])； dp[i][1] = max(dp[i-1][0]+prices[i]-fee, dp[i-1][1])
初始条件：dp[0][0], dp[0][1] = -prices[0], 0
返回值：max(dp[lens-1][0], dp[lens-1][1])"""
class Solution:
    def maxProfit(self, prices: List[int], fee: int) -> int:
        lens = len(prices)
        dp = [[0]*2 for _ in range(lens)]
        dp[0][0], dp[0][1] = -prices[0], 0
        for i in range(1, lens):
            dp[i][0] = max(dp[i-1][1]-prices[i], dp[i-1][0])
            dp[i][1] = max(dp[i-1][0]+prices[i]-fee, dp[i-1][1])
        return max(dp[lens-1][0], dp[lens-1][1])

In [ ]:
""" 这一题就解决了，但是这样处理 base case 很麻烦，而且注意一下状态转移方程，新状态只和相邻的一个状态有关，其实不用整个 dp 数组，只需要一个变量储存相邻的那个状态就足够了，这样可以把空间复杂度降到 O(1):"""
class Solution:
    def maxProfit(self, prices: List[int], fee: int) -> int:
        lens = len(prices)
        dp_0, dp_1 = -prices[0], 0
        for i in range(1, lens):
            temp = dp_0
            dp_0 = max(dp_1-prices[i], dp_0)
            dp_1 = max(temp+prices[i]-fee, dp_1)
        return max(dp_0, dp_1)

### 1024. 视频拼接
你将会获得一系列视频片段，这些片段来自于一项持续时长为 T 秒的体育赛事。这些片段可能有所重叠，也可能长度不一。
视频片段 clips[i] 都用区间进行表示：开始于 clips[i][0] 并于 clips[i][1] 结束。我们甚至可以对这些片段自由地再剪辑，例如片段 [0, 7] 可以剪切成 [0, 1] + [1, 3] + [3, 7] 三部分。
我们需要将这些片段进行再剪辑，并将剪辑后的内容拼接成覆盖整个运动过程的片段（[0, T]）。返回所需片段的最小数目，如果无法完成该任务，则返回 -1 。
示例 1：
输入：clips = [[0,2],[4,6],[8,10],[1,9],[1,5],[5,9]], T = 10
输出：3
解释：
我们选中 [0,2], [8,10], [1,9] 这三个片段。
然后，按下面的方案重制比赛片段：
将 [1,9] 再剪辑为 [1,2] + [2,8] + [8,9] 。
现在我们手上有 [0,2] + [2,8] + [8,10]，而这些涵盖了整场比赛 [0, 10]。

In [ ]:
"""动态规划DP: O(T*N), O(T), 但是merge sort需要O(NlogN), O(N), 所以总的复杂度为O(T*N+NlogN), O(T+N)
这道题我看到了懵逼了一些时间。思考了良久发现这是一个跳蛙类型的动态规划。
动态规划的要点：
DP数组当中存储的状态i是：0-i时间段当中需要最小的视频数量。
DP数组当中的简要状态方程：dp[j]=min(dp[j], dp[clips[i][0]]+1) if j in clips[i]的区间"""
class Solution:
    def videoStitching(self, clips: List[List[int]], T: int) -> int:
        self.mergeSort2D(clips)
        if clips[0][0] > 0 or clips[-1][-1] < T:
            return -1
        dp = [float("inf")]*(T+1)
        dp[0] = 0
        for t in range(1, T+1):
            for i in range(len(clips)):
                if t > clips[i][0] and t <= clips[i][1]:  # 如果t在clips[i]的区间内的话，clips[i]可以贡献一波
                    dp[t] = min(dp[t], dp[clips[i][0]]+1)
        return -1 if dp[T] == float("inf") else dp[T]  
        
    # 经典的merge sort实现方法必须掌握,O(NlogN),O(N)
    def mergeSort(self, arr):
        if len(arr) <= 1:
            return
        mid = len(arr)//2
        leftArr = arr[:mid]
        rightArr = arr[mid:]

        self.mergeSort(leftArr)
        self.mergeSort(rightArr)

        i, j, k = 0, 0, 0
        while i < len(leftArr) and j < len(rightArr):
            if leftArr[i] <= rightArr[j]:
                arr[k] = leftArr[i]
                i += 1
                k += 1
            else:
                arr[k] = rightArr[j]
                j += 1
                k += 1

        if i == len(leftArr):
            arr[k:] = rightArr[j:]
        if j == len(rightArr):
            arr[k:] = leftArr[i:mid]

    # merge sort a 2D array by the order of the first element of each subarry
    def mergeSort2D(self, arr2D):
        if len(arr2D) <= 1:
            return 
        mid = len(arr2D)//2
        leftArr2D = arr2D[:mid]
        rightArr2D = arr2D[mid:]
        self.mergeSort2D(leftArr2D)
        self.mergeSort2D(rightArr2D)
        i, j, k = 0, 0, 0
        while i < len(leftArr2D) and j < len(rightArr2D):
            if leftArr2D[i][0] <= rightArr2D[j][0]:
                arr2D[k] = leftArr2D[i]
                i += 1
                k += 1
            else:
                arr2D[k] = rightArr2D[j]
                j += 1
                k += 1
        if i == len(leftArr2D):
            arr2D[k:] = rightArr2D[j:]
        if j == len(rightArr2D):
            arr2D[k:] = leftArr2D[i:mid]

In [ ]:
"""解法二：贪心算法。这道题和算法书上的活动选择问题基本一致。O(NlogN+N), O(N)
利用贪心算法：在开始时间不大于t的视频中选择一个结束时间最大的那一个视频，其中t是上一个选择视频的结束时间。
算法：
对视频进行升序排列
选择一个从零开始的视频，它的结束时间是所有从零开始的视频中最大的，赋值给t
然后在其他视频中选择一个开始时间不超过t的视频，这个被选择的视频的结束时间也应该是最大的，赋值给t
重复，直到所有的视频都被遍历，或者t已经大于等于T"""
class Solution:
    def videoStitching(self, clips: List[List[int]], T: int) -> int:
        self.mergeSort2D(clips)
        print(clips)
        if clips[0][0] > 0 or clips[-1][-1] < T:
            return -1
        i, t, res = 0, 0, 0
        while i < len(clips) and t < T:
            temp_t = t
            while i < len(clips) and clips[i][0] <= temp_t:
                t = max(t, clips[i][1])
                i += 1
            res += 1
            print(i, t)
            if i < len(clips) and t < T and clips[i][0] > t:   # 因为数组排序了，clips[i][0] > t的话说明后面没有区间接的上了，如[[0,2],[7,10]]，这里的7就大于2。i < len(clips) and t < T 是必要的，因为可能i out of index, or t > T的情况下误输出-1
                return -1
        return res if t >= T else -1    # 注意这里不能用t==T，因为t可能大于T的。

### 91. 解码方法
一条包含字母 A-Z 的消息通过以下方式进行了编码：
'A' -> 1
'B' -> 2
...
'Z' -> 26
给定一个只包含数字（假设不包括0）的非空字符串，请计算解码方法的总数。
输入: "226"
输出: 3
解释: 它可以解码为 "BZ" (2 26), "VF" (22 6), 或者 "BBF" (2 2 6) 。

In [ ]:
"""经典的dp题，很容易想到。"""
class Solution:
    def numDecodings(self, s: str) -> int:
        lens = len(s)
        if lens <= 1:
            return lens
        dp = [0]*lens
        dp[0] = 1
        dp[1] = 1 if int(s[:1]) > 26 else 2
        for i in range(2, lens):
            if int(s[i-1:i]) <= 26:
                dp[i] = dp[i-1] + dp[i-2]
            else:
                dp[i] = dp[i-1]
        return dp[lens-1]

### 1155.掷骰子的N种方法
这里有 d 个一样的骰子，每个骰子上都有 f 个面，分别标号为 1, 2, ..., f。
我们约定：掷骰子的得到总点数为各骰子面朝上的数字的总和。
如果需要掷出的总点数为 target，请你计算出有多少种不同的组合情况（所有的组合情况总共有 f^d 种），模 10^9 + 7 后返回。

In [ ]:
""" dp[i][j]表示用i个骰子得出j的组合数
    dp[i][j] = dp[i-1][j-1] + ... + dp[i-1][j-f] if (j-f >= 0)  
    可以优化为1维，因为每次更新都只与上一次相关
    注意取模返回"""
class Solution:
    def numRollsToTarget(self, d: int, f: int, target: int) -> int:
        dp = [[0]*(target+1) for _ in range(d+1)]
        # 初始化条件，只有1个骰子的时候，1->f的方法均只有1种
        for i in range(1, target+1):   # 去他妈的检查了一个小时，原来就是这里错写成了range(target+1)
            if i <= f:
                dp[1][i] = 1
        for i in range(1, d+1):
            for j in range(1, target+1):
                for k in range(1, min(f, j)+1):
                    dp[i][j] += dp[i-1][j-k]   # 用i-1个骰子实现j-k，然后用剩下的一个实现j-(j-k)=k
                    dp[i][j] %= (10**9+7)
        return dp[d][target]
    
# 可优化为1维，不太理解！

### 279. 完全平方数
给定正整数 n，找到若干个完全平方数（比如 1, 4, 9, 16, ...）使得它们的和等于 n。你需要让组成和的完全平方数的个数最少。输入: n = 13
输出: 2
解释: 13 = 4 + 9.

In [ ]:
"""grandyang DP: dp[i]表示组成i的完全平方数的最小个数
递推公式：dp[i+j**2] = dp[i]+1
初始值：dp[1] = 1;  返回值：dp[n+1]"""
class Solution:
    def numSquares(self, n: int) -> int:
        dp = [5] * (n+1)   # 四平方和定理：任意一个正整数均可表示为4个整数的平方和，其实是可以表示为4个以内的平方数之和，那么就是说返回结果只有 1,2,3 或4其中的一个
        dp[0], dp[1] = 0, 1   # 这里必须从dp[0]=0开始算，不然dp[4]就无法等于1了！
        for i in range(0, n+1):
            j = 1
            while i+j**2 <= n:
                dp[i+j**2] = min(dp[i+j**2], dp[i] + 1)
                j += 1
        return dp[-1]

### 221. 最大正方形
在一个由 0 和 1 组成的二维矩阵内，找到只包含 1 的最大正方形，并返回其面积。

示例:

输入: 

1 0 1 0 0
1 0 1 1 1
1 1 1 1 1
1 0 0 1 0

输出: 4

In [ ]:
"""dp[i][j] 表示到在 (i, j) 位置所能组成的最大正方形的边长。返回max(dp)
初始条件是边角上的dp[i][j] = int(matrix[i][j])
只有当前 (i, j) 位置为1，dp[i][j] 才有可能大于0，否则 dp[i][j] 一定为0。当 (i, j) 位置为1，此时要看 dp[i-1][j-1], dp[i][j-1]，和 dp[i-1][j] 这三个位置，我们找其中最小的值，并加上1，就是 dp[i][j] 的当前值了，这个并不难想，毕竟不能有0存在，所以只能取交集，最后再用 dp[i][j] 的值来更新结果 res 的值即可。"""
class Solution:
    def maximalSquare(self, matrix: List[List[str]]) -> int:
        if not matrix:
            return 0
        len_row, len_col = len(matrix), len(matrix[0])
        res = 0
        dp = [[0]*len_col for _ in range(len_row)]
        for i in range(len_row):
            for j in range(len_col):
                if i==0 or j==0:
                    dp[i][j] = int(matrix[i][j])
                else:
                    if matrix[i][j] == "1":
                        dp[i][j] = 1 + min(dp[i-1][j-1], dp[i-1][j], dp[i][j-1])
                res = max(res, dp[i][j])
        return res**2

### 486. Predict the Winner
给定一个表示分数的非负整数数组。 玩家1从数组任意一端拿取一个分数，随后玩家2继续从剩余数组任意一端拿取分数，然后玩家1拿，……。每次一个玩家只能拿取一个分数，分数被拿取之后不再可取。直到没有剩余分数可取时游戏结束。最终获得分数总和最多的玩家获胜。给定一个表示分数的数组，预测玩家1是否会成为赢家。你可以假设每个玩家的玩法都会使他的分数最大化。

In [ ]:
"""时间复杂度：O(2^N)，其中 N 是数组的长度。空间复杂度：O(N)，为递归时栈使用的空间。"""
class Solution:
    def PredictTheWinner(self, nums: List[int]) -> bool:
        sumNums = sum(nums)
        firstMax = self._getMax(nums, 0, len(nums)-1)
        return firstMax >= sumNums-firstMax

    def _getMax(self, nums, i, j) -> int:
        if i == j:   # if there are odd number os elements
            return nums[i]
        elif i+1 == j:
            return max(nums[i], nums[j])
        
        # 都是聪明人，A如果想要赢A就取能保障自己赢的那个数让自己收益更大，这就是为什么大括号用max，A取完之后B来取，B也想要赢，于是B取剩下的数组中的能保证自己赢的那个数，于是B在剩余数组中取走属于自己的那份max，这样留给A下次来取的就是min了，这就是为什么小括号里用min
        return max(
            nums[i] + min(self._getMax(nums, i+1, j-1), self._getMax(nums, i+2, j)), 
            nums[j] + min(self._getMax(nums, i+1, j-1), self._getMax(nums, i, j-2))    )

In [ ]:
"""方法二：动态规划    时间复杂度：O(N^2); 空间复杂度：O(N^2)
我们同样可以使用动态规划来解决这个问题。用 dp(i, j) 表示当剩下的数为 nums[i .. j] 时，当前操作的选手（注意，不一定是先手）与另一位选手最多的分数差。当前操作的选手可以选择 nums[i] 并留下 nums[i+1 .. j]，或选择 nums[j] 并留下 nums[i .. j-1]，因此状态转移方程为：
dp(i, j) = max(nums[i] - dp(i+1, j), nums[j] - dp(i, j-1))， dp(i,j)依赖前面的值，前面的值分别来自下面一行和左边一列，因此是往二维数组的右上方移动，直到移动到第零行，如果 dp(0, n - 1) >= 0，那么先手必胜。
初始条件为dp(i, i) = nums[i]
https://leetcode-cn.com/problems/predict-the-winner/solution/san-chong-dpsi-lu-jie-jue-duo-si-lu-by-a-fei-8/"""
class Solution:
    def PredictTheWinner(self, nums: List[int]) -> bool:
        lens = len(nums)
        dp = [[0]*lens for _ in range(lens)]
        for i in range(lens):
            dp[i][i] = nums[i]
        for row in range(lens-1, -1, -1):
            for col in range(row+1, lens):
                dp[row][col] = max(nums[row]-dp[row+1][col], nums[col]-dp[row][col-1])
        return dp[0][lens-1] >= 0

### 983. 最低票价
在一个火车旅行很受欢迎的国度，你提前一年计划了一些火车旅行。在接下来的一年里，你要旅行的日子将以一个名为 days 的数组给出。每一项是一个从 1 到 365 的整数。
火车票有三种不同的销售方式：
一张为期一天的通行证售价为 costs[0] 美元；
一张为期七天的通行证售价为 costs[1] 美元；
一张为期三十天的通行证售价为 costs[2] 美元。
通行证允许数天无限制的旅行。 例如，如果我们在第 2 天获得一张为期 7 天的通行证，那么我们可以连着旅行 7 天：第 2 天、第 3 天、第 4 天、第 5 天、第 6 天、第 7 天和第 8 天。
返回你想要完成在给定的列表 days 中列出的每一天的旅行所需要的最低消费。
示例 1：
输入：days = [1,4,6,7,8,20], costs = [2,7,15]
输出：11
解释： 
例如，这里有一种购买通行证的方法，可以让你完成你的旅行计划：
在第 1 天，你花了 costs[0] = $2 买了一张为期 1 天的通行证，它将在第 1 天生效。
在第 3 天，你花了 costs[1] = $7 买了一张为期 7 天的通行证，它将在第 3, 4, ..., 9 天生效。
在第 20 天，你花了 costs[0] = $2 买了一张为期 1 天的通行证，它将在第 20 天生效。
你总共花了 $11，并完成了你计划的每一天旅行。

In [ ]:
"""是一个简单的动态规划问题。我们定义函数f(i)表示第i天的最低消费，那么
某天，如果你不必出行的话，等一等再购买火车票一定更优，如果你需要出行的话，那么就有三种选择：在通行期为 1 天、7 天、30 天中的火车票中选择一张购买。
f(i)=f(i−1)  if i not in days
else f(i)=min(f(i−1)+costs[0],f(i−7)+costs[1],f(i−30)+costs[2])"""
class Solution:
    def mincostTickets(self, days: List[int], costs: List[int]) -> int:
        lens = len(days)
        if lens == 0:
            return 0
        dp = [0] * 366
        days_set = set(days)
        for i in range(1, 366):
            if i not in days_set:
                dp[i] = dp[i-1]
            else:
                dp[i] = min(dp[i-1]+costs[0], dp[i-7]+costs[1], dp[i-30]+costs[2])
        return dp[days[-1]]

### 688. “马”在棋盘上的概率
已知一个 NxN 的国际象棋棋盘，棋盘的行号和列号都是从 0 开始。即最左上角的格子记为 (0, 0)，最右下角的记为 (N-1, N-1)。 
现有一个 “马”（也译作 “骑士”）位于 (r, c) ，并打算进行 K 次移动。 
如下图所示，国际象棋的 “马” 每一步先沿水平或垂直方向移动 2 个格子，然后向与之相垂直的方向再移动 1 个格子，共有 8 个可选的位置。
现在 “马” 每一步都从可选的位置（包括棋盘外部的）中独立随机地选择一个进行移动，直到移动了 K 次或跳到了棋盘外面。
求移动结束后，“马” 仍留在棋盘上的概率。
示例：
输入: 3, 2, 0, 0
输出: 0.0625
解释: 
输入的数据依次为 N, K, r, c
第 1 步时，有且只有 2 种走法令 “马” 可以留在棋盘上（跳到（1,2）或（2,1））。对于以上的两种情况，各自在第2步均有且只有2种走法令 “马” 仍然留在棋盘上。
所以 “马” 在结束后仍在棋盘上的概率为 0.0625。

In [ ]:
"""上论坛看大家的解法，结果发现都是换了一个角度来解决问题的，并不很关心骑士的起始位置，而是把棋盘上所有位置上经过K步还留在棋盘上的走法总和都算出来，那么最后直接返回需要的值即可。跟之前那道Out of Boundary Paths没啥本质上的区别，又是换了一个马甲就不会了系列。还是要用DP来做，我们可以用三维DP数组，也可以用二维DP数组来做，这里为了省空间，我们就用二维DP数组来做，其中dp[i][j]表示在棋盘(i, j)位置上走完当前步数还留在棋盘上的走法总和(注意是走法，不是步数)，初始化为1，我们其实将步骤这个维度当成了时间维度在不停更新。好，下面我们先写出8种‘日’字走法的位置的坐标，就像之前遍历迷宫上下左右四个方向坐标一样，这不过这次位置变了而已。然后我们一步一步来遍历，每一步都需要完整遍历一遍棋盘的每个位置，新建一个临时数组t，大小和dp数组相同，但是初始化为0，然后对于遍历到的棋盘上的每一个格子，我们都遍历8中解法，如果新的位置不在棋盘上了，直接跳过，否则就加上上一步中的dp数组中对应的值，遍历完棋盘后，将dp数组更新为这个临时数组t"""
class Solution:
    def knightProbability(self, N: int, K: int, r: int, c: int) -> float:
        if K == 0:
            return 1
        dp = [[1]*N for _ in range(N)]
        steps = [[1,2], [1,-2], [-1,2], [-1,-2], [2,1], [2,-1], [-2,1], [-2,-1]]
        
        # 走K次，就做K张路径表，每次路径表cur只跟前一次的路径表pre有关
        for k in range(K):
            temp = [[0]*N for _ in range(N)]
            for i in range(N):
                for j in range(N):
                    for step in steps:
                        x, y = i+step[0], j+step[1]
                        if 0 <= x < N and 0 <= y < N:
                            temp[x][y] += dp[i][j]   # temp[i][j] += dp[x][y]也可以，只是没那么好理解！
            dp = temp
        return dp[r][c]/(8**K)